In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import glob

## ETL: Transforms station data into one single csv

In [2]:
stations_centercoast_csv = 'Stations_CenterCoast.csv'
stations_centercoast_df = pd.read_csv(stations_centercoast_csv, index_col=0)

stations_coloradoriver_csv = 'Stations_ColoradoRiver.csv'
stations_coloradoriver_df = pd.read_csv(stations_coloradoriver_csv, index_col=0)

stations_northcoast_csv = 'Stations_NorthCoast.csv'
stations_northcoast_df = pd.read_csv(stations_northcoast_csv, index_col=0)

stations_northlahontan_csv = 'Stations_NorthLahontan.csv'
stations_northlahontan_df = pd.read_csv(stations_northlahontan_csv, index_col=0)

stations_sacramentoriver_csv = 'Stations_SacramentoRiver.csv'
stations_sacramentoriver_df = pd.read_csv(stations_sacramentoriver_csv, index_col=0)

stations_sanfranciscobay_csv = 'Stations_SanFranciscoBay.csv'
stations_sanfranciscobay_df = pd.read_csv(stations_sanfranciscobay_csv, index_col=0)

stations_sanjoaquin_csv = 'Stations_SanJoaquinRiver.csv'
stations_sanjoaquin_df = pd.read_csv(stations_sanjoaquin_csv, index_col=0)

stations_southcoast_csv = 'Stations_SouthCoast.csv'
stations_southcoast_df = pd.read_csv(stations_southcoast_csv, index_col=0)

stations_southlahontan_csv = 'Stations_SouthLahontan.csv'
stations_southlahontan_df = pd.read_csv(stations_southlahontan_csv, index_col=0)

stations_tularelake_csv = 'Stations_TulareLake.csv'
stations_tularelake_df = pd.read_csv(stations_tularelake_csv, index_col=0)

frames = [stations_centercoast_df, stations_coloradoriver_df, stations_northcoast_df, stations_northlahontan_df, 
          stations_sacramentoriver_df, stations_sanfranciscobay_df, stations_sanjoaquin_df,
         stations_southcoast_df, stations_southlahontan_df, stations_tularelake_df]

result = pd.concat(frames)
result.to_csv('Stations_all.csv', index=True)
result.head()

,Station Name,River Basin,County,Longitude,Latitude,ElevationFeet,Operator,HydrologicArea
ID,,,,,,,,
APT,APTOS CREEK AT RIDER ROAD,SAN LORENZO RIVER,SANTA CRUZ,-121.852800,37.036100,"1,123",Santa Cruz County,Central Coast
ARG,ARROYO GRANDE,ARROYO GRANDE CREEK,SAN LUIS OBISPO,-120.417000,35.117000,600,CA Dept of Forestry and Fire Protection,Central Coast
ARY,ARROYO SECO,SALINAS RIVER,MONTEREY,-121.488000,36.230000,980,US Forest Service,Central Coast
ASS,ARROYO SECO NR SOLEDAD,SALINAS RIVER,MONTEREY,-121.322708,36.280521,399,US Geological Survey,Central Coast
ATN,LAKE SAN ANTONIO,SALINAS RIVER,MONTEREY,-120.883000,35.798000,779,Monterey County,Central Coast


## ETL: Monthly Rainfall (2000-present)
Note: running this block of code takes approx 2 min.

In [3]:
# Create dataframe to hold information 
rainfall_monthly_df = pd.DataFrame(columns=['ID', 'year', 'month', 'monthly_rainfall'])

for yr in range(2000, 2024):
    year_str = str(yr)
    #the actual URL of the resource you want to retrieve
    url = 'https://cdec.water.ca.gov/reportapp/javareports?name=PRECIPMON.'+year_str
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content with Beautiful Soup
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the table using the 'find' method
        body_list = soup.find_all('body')
        table_body = body_list[2]
        table = table_body.find('table')
        # grab each row of the table 
        for row in table.find_all('tr'):
            # get the monthly rainfalls
            row_elems = row.find_all('td')
            if (len(row_elems) > 12):   # dont include the non data rows 
                id = row_elems[0].text.strip()
                year = year_str
                for i in range(2, len(row_elems)):
                    month = str(i-1)
                    m_rain = row_elems[i].text.strip()
                    row_data = {'ID': id, 'year': year, 'month': month, 'monthly_rainfall': m_rain}
                    rainfall_monthly_df = rainfall_monthly_df._append(row_data, ignore_index=True)
    else:
        print(f"Failed to retrieve content. Status code: {response.status_code}")

rainfall_monthly_df.to_csv('Rainfall_Monthly.csv', index=True)
rainfall_monthly_df.head(12)

,ID,year,month,monthly_rainfall
0,ALT,2000,1,.77
1,ALT,2000,2,.33
2,ALT,2000,3,.51
3,ALT,2000,4,2.10
4,ALT,2000,5,.89
5,ALT,2000,6,1.40
6,ALT,2000,7,.77
7,ALT,2000,8,1.81
8,ALT,2000,9,.08
9,ALT,2000,10,.41


## ETL: Daily Rainfall, Temperature, & Snowfall (2000 - present)

This section of code may potentially run for 7+ hours. 

In [39]:
# create dataframes to store daily precip data 
rainfall_daily_df = pd.DataFrame(columns=['ID', 'DATE / TIME (PST)', 'PPT INC INCHES', 'RAIN INCHES'])
snow_daily_df = pd.DataFrame(columns=['ID', 'DATE / TIME (PST)', 'SNOW WC INCHES', 'SNO ADJ INCHES', 'SNOW DP INCHES'])
temp_daily_df = pd.DataFrame(columns=['ID', 'DATE / TIME (PST)', 'TEMP MN DEG F', 'TEMP MX DEG F', 'TEMP AV DEG F'])

# get list of daily rainfall stations 
daily_ppt_stations_df = pd.read_csv('Daily_ppt_stations.csv', header=0)
station_id_list = daily_ppt_stations_df['ID'].tolist()

# loop over daily rainfall stations
for station in station_id_list[528:]:
    # loop over years
    for yr in range(2000, 2024):
        url = 'https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=' + station + '&end=' + str(yr) + '-12-01&span=365days'
        print(url)
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content with Beautiful Soup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the table
            table_list = soup.find_all('table')
        
            #check if that station and date has data 
            if (len(table_list) == 0):
                # do nothing 
                print("This year, or this station name has no data")
            else:
                table = soup.find_all('table')[0]
                header_elements = table.find_all('th')
    
                # get the headers / data column names 
                header_list = []
                for header in header_elements:
                    header_string = header.text.strip()
                    if (len(header.text.strip()) > 0):
                        header_list.append(header_string)
                header_list = header_list[1:]   # remove the date time header from this
    
                # create a temp df from the header list 
                temp_df = pd.DataFrame(columns=header_list)
    
                for row in table.find_all('tr'):
                    row_elems = row.find_all('td')
                    # create dictionary (pairwise col name and data)
                    temp_dict = {}
                    if (len(row_elems) > 0):
                        i = 1
                        for header in header_list[1:]:
                            temp_dict[header] = row_elems[i].text.strip()
                            i += 2
                        all_val_null = all(value == '--' for value in temp_dict.values())
                        if (not all_val_null):
                            temp_dict['ID'] = station
                            temp_dict['DATE / TIME (PST)'] = row_elems[0].text.strip()
                            temp_df = temp_df._append(temp_dict, ignore_index=True)
                ### display(temp_df)
    
                # copy over rain data 
                
                columns_to_copy = temp_df.filter(regex='RAIN|PPT', axis=1).columns.tolist()
                if (len(columns_to_copy) > 0 and temp_df.shape[0] > 0):
                    columns_to_copy.append('ID')
                    columns_to_copy.append('DATE / TIME (PST)')
                    rainfall_daily_df = pd.concat([rainfall_daily_df, temp_df[columns_to_copy]], ignore_index=True, sort=False)
    
                # copy over snow data
                columns_to_copy = temp_df.filter(regex='SNO', axis=1).columns.tolist()
                if (len(columns_to_copy) > 0 and temp_df.shape[0] > 0):
                    columns_to_copy.append('ID')
                    columns_to_copy.append('DATE / TIME (PST)')
                    snow_daily_df = pd.concat([snow_daily_df, temp_df[columns_to_copy]], ignore_index=True, sort=False)
    
                # copy over temperature data
                columns_to_copy = temp_df.filter(regex='TEMP', axis=1).columns.tolist()
                if (len(columns_to_copy) > 0 and temp_df.shape[0] > 0):
                    # then add relevant data and save 
                    columns_to_copy.append('ID')
                    columns_to_copy.append('DATE / TIME (PST)')
                    temp_daily_df = pd.concat([temp_daily_df, temp_df[columns_to_copy]], ignore_index=True, sort=False) 
        else:
            print(f"Failed to retrieve content. Status code: {response.status_code}")

https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2000-12-01&span=365days
This year, or this station name has no data
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2001-12-01&span=365days
This year, or this station name has no data
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2002-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2003-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2004-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2005-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2006-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2007-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2008-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2009-12-01&span=365days
https://cdec.water.ca.gov/dynamicapp/QueryDaily?s=BPT&end=2010-12-01&span=365days
https://cd

In [40]:
rainfall_daily_df.to_csv('Rainfall_Daily_.csv', index=True)
snow_daily_df.to_csv('Snowfall_Daily_.csv', index=True)
temp_daily_df.to_csv('Temperature_Daily_.csv', index=True)

## ETL: Combining Daily Data Into Single CSV File

In [44]:
# combine any temperature files into one 
# Specify the path pattern for your CSV files
file_pattern = 'Temperature_Daily*.csv'

# Get a list of all CSV files that match the pattern
files = glob.glob(file_pattern)
print(files)

combined_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file, low_memory=False)
    combined_df = combined_df._append(df, ignore_index=True)

combined_df.to_csv('Temperature_Daily.csv', index=False)


['Temperature_Daily_RDO.csv', 'Temperature_Daily_FO2.csv', 'Temperature_Daily_PLM.csv', 'Temperature_Daily_BGR.csv', 'Temperature_Daily_Total.csv', 'Temperature_Daily_CVM.csv', 'Temperature_Daily_SPS.csv', 'Temperature_Daily_HKY.csv', 'Temperature_Daily_ODA.csv', 'Temperature_Daily_WVR.csv', 'Temperature_Daily_API.csv', 'Temperature_Daily_CHI.csv']


In [3]:
# combine any snowfall files into one 
# Specify the path pattern for your CSV files
file_pattern = 'Snowfall_Daily*.csv'

# Get a list of all CSV files that match the pattern
files = glob.glob(file_pattern)
print(files)

combined_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file, low_memory=False)
    combined_df = combined_df._append(df, ignore_index=True)

combined_df.to_csv('Snowfall_Daily.csv', index=False)

['Snowfall_Daily_CHI.csv', 'Snowfall_Daily_ODA.csv', 'Snowfall_Daily_API.csv', 'Snowfall_Daily_WVR.csv', 'Snowfall_Daily_CVM.csv', 'Snowfall_Daily_BGR.csv', 'Snowfall_Daily_HIG.csv', 'Snowfall_Daily_SPS.csv', 'Snowfall_Daily_HKY.csv', 'Snowfall_Daily_PLM.csv', 'Snowfall_Daily_RDO.csv', 'Snowfall_Daily_FO2.csv']


In [4]:
# combine any rainfall files into one 
# Specify the path pattern for your CSV files
file_pattern = 'Rainfall_Daily*.csv'

# Get a list of all CSV files that match the pattern
files = glob.glob(file_pattern)
print(files)

combined_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file, low_memory=False)
    combined_df = combined_df._append(df, ignore_index=True)

combined_df.to_csv('Rainfall_Daily.csv', index=False)

['Rainfall_Daily_CHI.csv', 'Rainfall_Daily_API.csv', 'Rainfall_Daily_WVR.csv', 'Rainfall_Daily_ODA.csv', 'Rainfall_Daily_HIG.csv', 'Rainfall_Daily_HKY.csv', 'Rainfall_Daily_SPS.csv', 'Rainfall_Daily_CVM.csv', 'Rainfall_Daily_BGR.csv', 'Rainfall_Daily_PLM.csv', 'Rainfall_Daily_FO2.csv', 'Rainfall_Daily_RDO.csv']


## ETL : Grabbing unknown station data

In [7]:
rain_df = pd.read_csv('Rainfall_Daily.csv')
station_df = pd.read_csv('Stations_all.csv')

# Find the unique 'id' values in both data frames
rain_ids = set(rain_df['ID'])
station_ids = set(station_df['ID'])

# Find the 'id' values in rain that do not have a corresponding station id
missing_ids_in_station = rain_ids - station_ids

print(missing_ids_in_station)

/var/folders/rj/9p72k2_n4n5dp_xbmdt7ygdc0000gn/T/ipykernel_5983/4149001459.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  rain_df = pd.read_csv('Rainfall_Daily.csv')


{'SNR', 'RBR', 'MCM', 'EWS', 'MDL', 'BVA', 'TAY', 'MCE', 'ROV', 'MDZ', 'ODL', 'OGC', 'TAL', 'FO2', 'TOK', 'MVD', 'CHI', 'SMV', 'GLR'}


In [26]:
station_df = pd.read_csv('Stations_all.csv')

for station_str in missing_ids_in_station:
    #the actual URL of the resource you want to retrieve
    url = 'https://cdec.water.ca.gov/dynamicapp/staMeta?station_id='+station_str
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content with Beautiful Soup
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the table using the 'find' method
        table_list = soup.find_all('table')
        table = table_list[0]
        list_of_values = []
        for tr in table.find_all('tr'):
            for td in tr.find_all('td'):
                list_of_values.append(td.text.strip())
        station_id = list_of_values[1]
        elevation = list_of_values[3].split()[0]
        riverbasin = list_of_values[5]
        county = list_of_values[7]
        hydroarea = list_of_values[9]
        lat = list_of_values[13].split('.')[0]
        long = list_of_values[15].split('.')[0]
        operator = list_of_values[17]
        heading = soup.find('h2').text.strip()
        row_data = {'ID': station_id, 'Station Name': heading, 'River Basin': riverbasin, 'County': county, 
                    'Longitude': long, 'Latitude': lat, 'ElevationFeet': elevation, 'Operator': operator,
                   'HydrologicArea': hydroarea}
        display(row_data)
        station_df = station_df._append(row_data, ignore_index=True)
    else:
        print(f"Failed to retrieve content. Status code: {response.status_code}")

station_df.tail(15)
station_df.to_csv('Stations_All.csv', index=True)
# Station Name	River Basin	County	Longitude	Latitude	ElevationFeet	Operator	HydrologicArea

{'ID': 'SNR',
 'Station Name': 'SANGUINETTI RANCH',
 'River Basin': 'SAN JOAQUIN RIVER',
 'County': 'SAN JOAQUIN',
 'Longitude': '-120',
 'Latitude': '37',
 'ElevationFeet': '238',
 'Operator': 'San Joaquin County',
 'HydrologicArea': 'SAN JOAQUIN RIVER'}

{'ID': 'RBR',
 'Station Name': 'ROBIDART RANCH',
 'River Basin': 'CALAVERAS RIVER',
 'County': 'SAN JOAQUIN',
 'Longitude': '-121',
 'Latitude': '38',
 'ElevationFeet': '315',
 'Operator': 'San Joaquin County',
 'HydrologicArea': 'SAN JOAQUIN RIVER'}

{'ID': 'MCM',
 'Station Name': 'MORRISON CREEK AT MACK ROAD',
 'River Basin': 'SACRAMENTO RIVER',
 'County': 'SACRAMENTO',
 'Longitude': '-121',
 'Latitude': '38',
 'ElevationFeet': '13',
 'Operator': 'Sacramento Dept of Public Works',
 'HydrologicArea': 'SACRAMENTO RIVER'}

{'ID': 'EWS',
 'Station Name': 'PLUMAS EUREKA ST PARK WEATHER STATION',
 'River Basin': 'FEATHER RIVER',
 'County': 'PLUMAS',
 'Longitude': '-120',
 'Latitude': '39',
 'ElevationFeet': '5107',
 'Operator': 'CA Dept of Water Resources/Flood Management',
 'HydrologicArea': 'SACRAMENTO RIVER'}

{'ID': 'MDL',
 'Station Name': 'MUD LAKE',
 'River Basin': 'MOKELUMNE RIVER',
 'County': 'AMADOR',
 'Longitude': '-120',
 'Latitude': '38',
 'ElevationFeet': '7900',
 'Operator': 'Sacramento Municipal Utility District',
 'HydrologicArea': 'SAN JOAQUIN RIVER'}

...

{'ID': 'TOK',
 'Station Name': 'THOUSAND OAKS RAWS',
 'River Basin': 'SANTA CLARA RIVER',
 'County': 'VENTURA',
 'Longitude': '-118',
 'Latitude': '34',
 'ElevationFeet': '795',
 'Operator': 'Ventura County Fire',
 'HydrologicArea': 'SOUTH COAST'}

{'ID': 'MVD',
 'Station Name': 'MUCK VALLEY DIVERSION (PG&E)',
 'River Basin': 'PIT RIVER',
 'County': 'SHASTA',
 'Longitude': '-121',
 'Latitude': '41',
 'ElevationFeet': '4300',
 'Operator': 'Pacific Gas & Electric',
 'HydrologicArea': 'SACRAMENTO RIVER'}

{'ID': 'CHI',
 'Station Name': 'CHICO',
 'River Basin': 'BUTTE CREEK',
 'County': 'BUTTE',
 'Longitude': '-121',
 'Latitude': '39',
 'ElevationFeet': '230',
 'Operator': 'CA Dept of Forestry and Fire Protection',
 'HydrologicArea': 'SACRAMENTO RIVER'}

{'ID': 'SMV',
 'Station Name': 'SIMI VALLEY',
 'River Basin': 'SANTA CLARA RIVER',
 'County': 'VENTURA',
 'Longitude': '-118',
 'Latitude': '34',
 'ElevationFeet': '914',
 'Operator': 'Ventura County',
 'HydrologicArea': 'CENTRAL COAST'}

{'ID': 'GLR',
 'Station Name': 'GILMORE RANCH',
 'River Basin': 'CALAVERAS RIVER',
 'County': 'SAN JOAQUIN',
 'Longitude': '-120',
 'Latitude': '38',
 'ElevationFeet': '220',
 'Operator': 'San Joaquin County',
 'HydrologicArea': 'SAN JOAQUIN RIVER'}